In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 

from src.normalisers import NormaliseListings, NormaliseReviews

In [ ]:
reviews = pd.read_csv('data/reviews.csv.gz')
listings = pd.read_csv('data/listings.csv.gz')
calendar = pd.read_csv('data/calendar.csv.gz')
wards = pd.read_csv('data/wards.csv')

In [3]:
# Normalise data
# Reviews
normalise_reviews = NormaliseReviews(reviews)

# Reviews tables
review_comments = normalise_reviews.normalise_reviews()
reviewers = normalise_reviews.normalise_reviewers()

#Listings
normalise_listings = NormaliseListings(listings)

# Listings tables

unique_hosts = normalise_listings.normalise_hosts()
unique_listings = normalise_listings.normalise_listings()
neighbourhoods = normalise_listings.normalise_neighbourhoods()
listing_reviews = normalise_listings.normalise_listing_reviews()
neighbourhood_overviews = normalise_listings.normalise_neighbourhood_overview()

# AirBnB Dashboard Plan

1. Divide the data according to ward (dropdown)
2. Show count of unique lisings (card)
3. Show average rating (card)
4. Show avarage price (card)
5. Total Listings (card)
6. Total Hosts (card) 
7. Percent Superhosts (card)
8. Average Response Rate (card)
9. Average Acceptance Rate (card)
10. Reviews Sentiment Analysis (wordcloud)



In [4]:

listings_x_ratings = pd.merge(
    unique_listings, listing_reviews, on='listing_id', how='left'
)  
listings_x_ratings.head()

,listing_id,scrape_id,name,description,host_id,latitude,longitude,property_type,room_type,accommodates,...,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,15007,20250928034929,Blaauwberg House on the beach in Bloubergstrand,Welcome to our self-catering beach-front famil...,59072,-33.800010,18.460630,Entire home,Entire home/apt,6,...,2013-12-15,2024-10-19,4.81,4.91,4.83,4.98,4.94,4.94,4.85,0.33
1,15068,20250928034929,Grande Bay,Modern spacious apartment. Three bedrooms two...,59318,-33.788260,18.459400,Entire rental unit,Entire home/apt,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15077,20250928034929,Relaxed beach living in style,Our Superior Rooms (approx. 25 m²) are located...,59342,-33.858356,18.490376,Private room in rental unit,Private room,2,...,2013-01-06,2022-06-16,5.00,4.86,4.86,4.86,4.71,4.86,5.00,0.05
3,15199,20250928034929,Self catering apartment,NaN,59694,-33.911150,18.412350,Entire rental unit,Entire home/apt,2,...,2014-03-04,2016-04-15,4.50,4.00,4.00,4.50,5.00,5.00,4.00,0.01
4,15354,20250928034929,"Aurora Self Catering Units, Durbanville, Cape ...",Durbanville is centrally located for daytrips ...,60196,-33.830740,18.631420,Entire guest suite,Entire home/apt,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
listings_x_ratings.dtypes

listing_id                       int64
scrape_id                        int64
name                            object
description                     object
host_id                          int64
latitude                       float64
longitude                      float64
property_type                   object
room_type                       object
accommodates                     int64
bathrooms                      float64
bathrooms_text                  object
bedrooms                       float64
beds                           float64
amenities                       object
price_usd                      Float64
estimated_occupancy_l365d        int64
estimated_revenue_l365d        float64
instant_bookable                  bool
neighbourhood_id                 int64
number_of_reviews                int64
number_of_reviews_ltm            int64
number_of_reviews_l30d           int64
number_of_reviews_ly             int64
first_review                    object
last_review              

## Ward Dropdown

In [6]:
wards['neighbourhood_id'] = wards['Name'].str.replace('Ward', '').str.strip().astype(int)   

new_wards = pd.merge(
    neighbourhoods,
    wards,
    on='neighbourhood_id',
    how='left'
)

new_wards.drop(columns=['neighbourhood_cleansed'], inplace=True)
new_wards.columns = [col.lower() for col in new_wards.columns]
new_wards.drop_duplicates(inplace=True)
new_wards['neighbourhood_id'] = new_wards['neighbourhood_id']
new_wards.sort_values('neighbourhood_id')


,neighbourhood_id,name,latitude,longitude
28,1,Ward 1,-33.878654,18.570032
6,2,Ward 2,-33.894026,18.602100
31,3,Ward 3,-33.892196,18.656065
1,4,Ward 4,-33.855849,18.514835
9,5,Ward 5,-33.870701,18.542310
...,...,...,...,...
67,111,Ward 111,-33.850344,18.724919
3,112,NaN,NaN,NaN
20,113,NaN,NaN,NaN
2,115,NaN,NaN,NaN


In [7]:
new_wards.loc[new_wards['name'].isna(), 'name'] = new_wards[new_wards['name'].isna()]['neighbourhood_id'].apply(lambda x: f"Ward {x}")

In [8]:
new_wards[new_wards['latitude'].isna()]

,neighbourhood_id,name,latitude,longitude
2,115,Ward 115,NaN,NaN
3,112,Ward 112,NaN,NaN
20,113,Ward 113,NaN,NaN
54,116,Ward 116,NaN,NaN


In [9]:
new_wards['neighbourhood_id'].dtype

dtype('int64')

In [10]:
unique_listings['neighbourhood_id'].dtype

dtype('int64')

In [11]:
# Get neighbourhood_ids with NaN latitude/longitude
nan_wards = new_wards[new_wards['latitude'].isna()]['neighbourhood_id'].unique().tolist()   

# For each ward with NaN coordinates, fill with mean from listings
for ward_id in nan_wards:
    #print("Filling coordinates for ward_id:", ward_id)
    mean_lat = unique_listings[unique_listings['neighbourhood_id'] == ward_id]['latitude'].mean()
    mean_lon = unique_listings[unique_listings['neighbourhood_id'] == ward_id]['longitude'].mean()
    #print("lat:", mean_lat, "lon:", mean_lon)
    new_wards.loc[new_wards['neighbourhood_id'] == ward_id, 'latitude'] = mean_lat
    new_wards.loc[new_wards['neighbourhood_id'] == ward_id, 'longitude'] = mean_lon

In [12]:
new_wards[new_wards['latitude'].isna()]

,neighbourhood_id,name,latitude,longitude


## Host Metrics

In [13]:
clean_hosts = unique_hosts.drop_duplicates(subset=['host_id'])
clean_hosts.drop(columns=['host_name', 'host_location', 'host_has_profile_pic', 'host_about', 'host_verifications'   ], inplace=True)

In [14]:
unique_hosts.head(10)

,host_id,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms
0,59072,Dirk,2009-12-01,"Cape Town, South Africa",Proud to live in the beautiful city of Cape To...,within a few hours,100.0,88.0,False,7.0,19.0,"['email', 'phone', 'work_email']",True,True,1,1,0,0
1,59318,Linda,2009-12-02,<NA>,<NA>,a few days or more,0.0,33.0,False,7.0,20.0,"['email', 'phone']",True,True,5,5,0,0
2,59342,Georg,2009-12-02,Gibraltar,challenge us please! we love to perform!,within a few hours,100.0,80.0,False,7.0,7.0,"['email', 'phone']",True,True,6,1,5,0
3,59694,Alexa,2009-12-03,"Cape Town, South Africa",Professional businesswoman dedicated to provid...,<NA>,<NA>,<NA>,False,2.0,2.0,"['email', 'phone']",True,True,1,1,0,0
4,60196,Ian,2009-12-05,"Cape Town, South Africa",<NA>,<NA>,<NA>,<NA>,False,1.0,2.0,"['email', 'phone']",True,False,1,1,0,0
5,60443,Jean,2009-12-06,"Betty's Bay, South Africa",Like to be as accommodating and informative a...,a few days or more,0.0,0.0,False,2.0,2.0,"['email', 'phone']",True,True,1,1,0,0
6,61441,Kevin,2009-12-09,"Cape Town, South Africa",<NA>,within a few hours,100.0,100.0,False,7.0,7.0,"['email', 'phone', 'work_email']",True,True,7,7,0,0
7,63255,Cheryl,2009-12-15,"Cape Town, South Africa",Own Self Catering cottages in Cape Town,within an hour,100.0,83.0,True,3.0,3.0,"['email', 'phone']",True,True,3,2,1,0
8,71221,Abe,2010-01-17,"Cape Town, South Africa",Businessman rom Cape Town,<NA>,<NA>,<NA>,False,1.0,1.0,"['email', 'phone']",True,False,1,0,1,0
9,73439,Beverley,2010-01-25,"Cape Town, South Africa",We live in a beautiful leafy suburb of Cape To...,within a few hours,100.0,84.0,True,5.0,5.0,"['email', 'phone']",True,True,5,5,0,0


In [15]:
unique_hosts.dtypes

host_id                                                  int64
host_name                                       string[python]
host_since                                      string[python]
host_location                                   string[python]
host_about                                      string[python]
host_response_time                              string[python]
host_response_rate                                     Float64
host_acceptance_rate                                   Float64
host_is_superhost                                         bool
host_listings_count                                    float64
host_total_listings_count                              float64
host_verifications                              string[python]
host_has_profile_pic                                      bool
host_identity_verified                                    bool
calculated_host_listings_count                           int64
calculated_host_listings_count_entire_homes            

In [21]:
total_hosts = unique_hosts.shape[0]
print("Total hosts:", total_hosts)

Total hosts: 14134


In [22]:
num_superhosts = unique_hosts['host_is_superhost'].sum()
print("Number of superhosts:", num_superhosts)
percent_superhosts = (num_superhosts / total_hosts) * 100
print("Percent superhosts:", percent_superhosts)

Number of superhosts: 3559
Percent superhosts: 25.18041601811235


In [18]:
ward = 1
filtered_listings = listings_x_ratings[listings_x_ratings['neighbourhood_id'] == ward]
filtered_listings.shape
filtered_listings.host_id.unique()

array([  9752740,  26564335,  43741993,  50715882,  50722620,  52787977,
        60981222,  56557141,  73885825,  85808883,  87188778,  87661242,
       104111522, 104749994, 105665540,  75764692,  43165399, 136156678,
        47435679, 148022639, 118995155, 153283820,  89839228, 155794491,
       158279250, 161555209,  84705605,  86243595, 172163084, 165445882,
       181021441, 184339098, 188474329, 190581790, 168133300, 132755819,
       229976526, 169476286, 240853629, 274676442, 279696219, 168389350,
       286152157, 186103957,  30248864, 121528558, 217179711, 254461565,
       220885221, 230352627, 341686124, 226054672,  72459979, 257121070,
       315264981, 371605160, 373569876, 392471309, 127854974, 222255161,
       399085675, 414303207, 119746967, 299992602, 402009049, 147173769,
       137118143, 432023798, 138954647, 187421610, 171972024,  45577453,
        30368501,  59108572, 104783163, 209176997, 464521098, 242208883,
       477071182, 442172606, 275726377, 211270409, 

In [19]:
unique_hosts[unique_hosts['host_id'].isin(filtered_listings['host_id'].unique())]

,host_id,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms
206,3961453,Host Agents,2012-10-24,<NA>,"We operate with guest's best interest in mind,...",within an hour,100.0,100.0,False,108.0,167.0,['phone'],True,True,100,100,0,0
209,9752740,Jane Christine Beatrix,2013-11-01,"Cape Town, South Africa",I am a South African lady married to a Portugu...,within an hour,100.0,100.0,True,2.0,4.0,"['email', 'phone']",True,True,2,2,0,0
720,26564335,Esther,2015-01-23,"Cape Town, South Africa",Welcome!!! I am waiting for you in Cape town....,<NA>,<NA>,<NA>,False,1.0,1.0,"['email', 'phone']",True,False,1,0,1,0
1194,43741993,Willem,2015-09-09,"Cape Town, South Africa",<NA>,<NA>,<NA>,<NA>,False,1.0,1.0,"['email', 'phone']",True,False,1,1,0,0
1613,50715882,Maree,2015-12-07,"Cape Town, South Africa","Are you looking for a family home, where your ...",<NA>,<NA>,<NA>,False,1.0,1.0,"['email', 'phone', 'work_email']",True,False,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24898,517318539,Giovanni Alexandre,2023-05-30,"Cape Town, South Africa",I’m a student who loves playing padel and bein...,<NA>,<NA>,<NA>,False,2.0,3.0,['phone'],True,True,2,2,0,0
24952,699111688,Micheal,2025-06-03,<NA>,<NA>,within an hour,100.0,75.0,False,1.0,1.0,"['email', 'phone']",False,True,1,1,0,0
25423,706217019,Ronell,2025-07-09,"Cape Town, South Africa",<NA>,within an hour,100.0,97.0,False,5.0,5.0,"['email', 'phone']",False,True,5,3,2,0
25427,703608224,Tshilidzi Adelphia,2025-06-26,<NA>,<NA>,<NA>,<NA>,<NA>,False,7.0,7.0,"['email', 'phone']",True,True,7,2,5,0


In [20]:
total_hosts = unique_hosts['host_id'].nunique()
mean_response_rate = unique_hosts['host_response_rate'].mean()
mean_acceptance_rate = unique_hosts['host_acceptance_rate'].mean()
super_hosts_count = unique_hosts['host_is_superhost'].sum()
verified_hosts_count = unique_hosts['host_identity_verified'].sum()  
print("Total hosts:", total_hosts)
print("Superhosts count:", super_hosts_count)
print("Verified hosts count:", verified_hosts_count)

get_percent = lambda x: x / total_hosts * 100 if total_hosts > 0 else 0
super_hosts_percent = get_percent(super_hosts_count)
verified_hosts_percent = get_percent(verified_hosts_count)


metrics = {     
            'total_hosts': total_hosts,
            'mean_response_rate': mean_response_rate,
            'mean_acceptance_rate': mean_acceptance_rate,
            'super_hosts_percent': super_hosts_percent,
            'verified_hosts_percent': verified_hosts_percent
        }

Total hosts: 14134
Superhosts count: 3559
Verified hosts count: 12891
